In [ ]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163765 sha256=fc4de0226753f84a795881c3b57ebb0496ef08460745c2f31a09acca7f6e3fe7
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
import pandas as pd

# Load the dataset
movies_metadata = pd.read_csv('/content/sample_data/movies_metadata.csv')
ratings = pd.read_csv('/content/sample_data/ratings_small.csv')

# Dataset description
print(movies_metadata.info())
print(ratings.info())


<ipython-input-3-b1e39db570c1>:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata = pd.read_csv('/content/sample_data/movies_metadata.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [ ]:
ratings.movieId = pd.to_numeric(ratings.movieId, errors='coerce')
ratings.userId = pd.to_numeric(ratings.userId, errors='coerce')
ratings.rating = pd.to_numeric(ratings.rating, errors='coerce')

len(ratings)
df = ratings
print(df)

        userId  movieId  rating   timestamp
0            1       31     2.5  1260759144
1            1     1029     3.0  1260759179
2            1     1061     3.0  1260759182
3            1     1129     2.0  1260759185
4            1     1172     4.0  1260759205
...        ...      ...     ...         ...
99999      671     6268     2.5  1065579370
100000     671     6269     4.0  1065149201
100001     671     6365     4.0  1070940363
100002     671     6385     2.5  1070979663
100003     671     6565     3.5  1074784724

[100004 rows x 4 columns]


In [ ]:
from surprise import Reader, Dataset, KNNBasic, SVD, NMF
from surprise.model_selection import GridSearchCV, cross_validate

reader = Reader(rating_scale=(0.5, 5.0))

data = Dataset.load_from_df( ratings[['userId', 'movieId', 'rating']], reader = reader )

In [ ]:
#SVD

algo_svd = SVD()
cross_validate(algo=algo_svd, data=data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9013  0.9033  0.8912  0.8948  0.8952  0.8972  0.0045  
Fit time          2.34    1.48    1.47    1.66    1.62    1.71    0.32    
Test time         0.25    0.13    0.21    0.11    0.26    0.19    0.06    


{'test_rmse': array([0.9013233 , 0.90332165, 0.8912006 , 0.89479572, 0.89521767]),
 'fit_time': (2.3443784713745117,
  1.477057695388794,
  1.474977970123291,
  1.6573431491851807,
  1.6193263530731201),
 'test_time': (0.2523932456970215,
  0.12500572204589844,
  0.20664763450622559,
  0.11356186866760254,
  0.26282215118408203)}

In [ ]:
#Fine Tuning SVD using GridSearchCV

param_grid = {'n_factors' : [50, 75], 'lr_all' : [0.5, 0.05], 'reg_all' : [0.06, 0.04]}

gs = GridSearchCV(algo_class=SVD, measures=['RMSE'], param_grid=param_grid)
gs.fit(data)

# Best RMSE score
print('Best Score :', gs.best_score['rmse'])

# Combination of parameters that gave the best RMSE score
print('Best Parameters :', gs.best_params['rmse'])


Best Score : 0.8889549479313089
Best Parameters : {'n_factors': 75, 'lr_all': 0.05, 'reg_all': 0.06}


In [ ]:
#SVD_train

from surprise.model_selection import train_test_split
import random
import numpy as np

random.seed(42)
np.random.seed(42)

# Split the data into train and test sets
trainset, testset = train_test_split(data, test_size=0.25)

# Instantiate the SVD algorithm with the best parameters
best_svd = SVD(n_factors=gs.best_params['rmse']['n_factors'],
               lr_all=gs.best_params['rmse']['lr_all'],
               reg_all=gs.best_params['rmse']['reg_all'])

# Train the model on the entire dataset
trainset = data.build_full_trainset()
best_svd.fit(trainset)

# Get a list of all movie IDs
all_movie_ids = set(ratings['movieId'].unique())

# Get a list of movie IDs rated by user 1
rated_movie_ids_by_user = set(ratings[ratings['userId'] == 1]['movieId'].unique())

# Movies that user 1 has not rated
unrated_movies = list(all_movie_ids - rated_movie_ids_by_user)

# Make predictions for unrated movies
predictions = [best_svd.predict(1, movie_id) for movie_id in unrated_movies]

# Sort predictions by estimated rating in descending order
top_recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)

# Print the top 10 recommended movie titles with available information
top_recommendations_with_info = []

for prediction in top_recommendations:
    movie_id = prediction.iid
    movie_info = movies_metadata[movies_metadata['id'] == str(movie_id)]
    if not movie_info.empty:
        top_recommendations_with_info.append(prediction)
        if len(top_recommendations_with_info) == 10:
            break

for i, prediction in enumerate(top_recommendations_with_info, start=1):
    movie_id = prediction.iid
    movie_title = movies_metadata[movies_metadata['id'] == str(movie_id)]['title'].values[0]
    print(f"{i}. {movie_title} (MovieID: {movie_id}, Predicted Rating: {prediction.est:.2f})")

1. Urban Explorer (MovieID: 73290, Predicted Rating: 3.85)
2. A Christmas Story (MovieID: 850, Predicted Rating: 3.79)
3. More of Me (MovieID: 134881, Predicted Rating: 3.77)
4. Gentlemen Prefer Blondes (MovieID: 759, Predicted Rating: 3.75)
5. The In-Laws (MovieID: 5146, Predicted Rating: 3.75)
6. License to Wed (MovieID: 2959, Predicted Rating: 3.73)
7. Design of Death (MovieID: 114464, Predicted Rating: 3.72)
8. The Crazy Stranger (MovieID: 5071, Predicted Rating: 3.68)
9. The Night of the Hunter (MovieID: 3112, Predicted Rating: 3.67)
10. Anybody's Son Will Do (MovieID: 38061, Predicted Rating: 3.66)


In [ ]:
#SVD_test

# Make predictions for unrated movies in the test set
test_predictions = [best_svd.predict(user_id, movie_id) for user_id, movie_id, _ in testset]

# Sort predictions by estimated rating in descending order
top_test_recommendations = sorted(test_predictions, key=lambda x: x.est, reverse=True)

# Print the top 10 recommended movie titles with available information for the test set
top_test_recommendations_with_info = []

for prediction in top_test_recommendations:
    movie_id = prediction.iid
    movie_info = movies_metadata[movies_metadata['id'] == str(movie_id)]
    if not movie_info.empty:
        top_test_recommendations_with_info.append(prediction)
        if len(top_test_recommendations_with_info) == 10:
            break

for i, prediction in enumerate(top_test_recommendations_with_info, start=1):
    movie_id = prediction.iid
    movie_title = movies_metadata[movies_metadata['id'] == str(movie_id)]['title'].values[0]
    print(f"{i}. {movie_title} (MovieID: {movie_id}, Predicted Rating: {prediction.est:.2f})")


1. Walk on Water (MovieID: 26, Predicted Rating: 5.00)
2. Lost Horizon (MovieID: 3598, Predicted Rating: 5.00)
3. Solaris (MovieID: 593, Predicted Rating: 5.00)
4. Irma la Douce (MovieID: 2690, Predicted Rating: 5.00)
5. Birdman of Alcatraz (MovieID: 898, Predicted Rating: 5.00)
6. The Big Sleep (MovieID: 910, Predicted Rating: 5.00)
7. Lonely Hearts (MovieID: 1252, Predicted Rating: 5.00)
8. Hard Target (MovieID: 2019, Predicted Rating: 5.00)
9. Confession of a Child of the Century (MovieID: 58559, Predicted Rating: 5.00)
10. Scarface (MovieID: 111, Predicted Rating: 5.00)


In [ ]:
from surprise import accuracy

# Calculate MSE for train set
train_predictions = best_svd.test(trainset.build_testset())
train_mse = accuracy.mse(train_predictions)
print(f"MSE for the train set(SVD): {train_mse}")

# Calculate MSE for test set
test_predictions = best_svd.test(testset)
test_mse = accuracy.mse(test_predictions)
print(f"MSE for the test set(SVD): {test_mse}")


MSE: 0.1814
MSE for the train set(SVD): 0.1814159494700053
MSE: 0.1823
MSE for the test set(SVD): 0.182318783341357


In [ ]:
#Non-Negative Matrix Factorization (NMF)

algo_nmf = NMF()
cross_validate(data=data, algo=algo_nmf, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9454  0.9592  0.9394  0.9420  0.9504  0.9473  0.0070  
Fit time          3.00    3.55    5.65    3.20    3.07    3.69    0.99    
Test time         0.33    0.12    0.18    0.39    0.11    0.23    0.11    


{'test_rmse': array([0.94540763, 0.95917567, 0.93943878, 0.9420275 , 0.95035367]),
 'fit_time': (3.001987934112549,
  3.549396514892578,
  5.645415544509888,
  3.2036197185516357,
  3.069304943084717),
 'test_time': (0.32621145248413086,
  0.12238931655883789,
  0.1818091869354248,
  0.390850305557251,
  0.1063385009765625)}

In [ ]:
#NMF_train

import random
import numpy as np
from surprise.model_selection import train_test_split

random.seed(42)
np.random.seed(42)

# Split the data into train and test sets
trainset, testset = train_test_split(data, test_size=0.25)

# Train the NMF model on the entire dataset
trainset = data.build_full_trainset()
algo_nmf.fit(trainset)

# Get a list of all movie IDs
all_movie_ids = set(ratings['movieId'].unique())

# Get a list of movie IDs rated by user 1
rated_movie_ids_by_user = set(ratings[ratings['userId'] == 1]['movieId'].unique())

# Movies that user 1 has not rated
unrated_movies = list(all_movie_ids - rated_movie_ids_by_user)

# Make predictions for unrated movies
predictions = [algo_nmf.predict(1, movie_id) for movie_id in unrated_movies]

# Sort predictions by estimated rating in descending order
top_recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)

# Print the top 10 recommended movie titles with available information
top_recommendations_with_info = []

for prediction in top_recommendations:
    movie_id = prediction.iid
    movie_info = movies_metadata[movies_metadata['id'] == str(movie_id)]
    if not movie_info.empty:
        top_recommendations_with_info.append(prediction)
        if len(top_recommendations_with_info) == 10:
            break

for i, prediction in enumerate(top_recommendations_with_info, start=1):
    movie_id = prediction.iid
    movie_title = movies_metadata[movies_metadata['id'] == str(movie_id)]['title'].values[0]
    print(f"{i}. {movie_title} (MovieID: {movie_id}, Predicted Rating: {prediction.est:.2f})")

1. End of the World (MovieID: 3030, Predicted Rating: 4.17)
2. The Vampire Lovers (MovieID: 31952, Predicted Rating: 4.09)
3. Gentlemen Prefer Blondes (MovieID: 759, Predicted Rating: 3.97)
4. Pulgasari (MovieID: 26974, Predicted Rating: 3.96)
5. Still Bill (MovieID: 55063, Predicted Rating: 3.94)
6. The Return of the King (MovieID: 1361, Predicted Rating: 3.93)
7. Veerana (MovieID: 98122, Predicted Rating: 3.91)
8. Murder in Three Acts (MovieID: 6107, Predicted Rating: 3.90)
9. Before Sunset (MovieID: 80, Predicted Rating: 3.85)
10. Strangers on a Train (MovieID: 845, Predicted Rating: 3.81)


In [ ]:
#NMF_test


# Make predictions for unrated movies in the test set
test_predictions = [algo_nmf.predict(user_id, movie_id) for user_id, movie_id, _ in testset]

# Sort predictions by estimated rating in descending order
top_test_recommendations = sorted(test_predictions, key=lambda x: x.est, reverse=True)

# Print the top 10 recommended movie titles with available information for the test set
top_test_recommendations_with_info = []

for prediction in top_test_recommendations:
    movie_id = prediction.iid
    movie_info = movies_metadata[movies_metadata['id'] == str(movie_id)]
    if not movie_info.empty:
        top_test_recommendations_with_info.append(prediction)
        if len(top_test_recommendations_with_info) == 10:
            break

for i, prediction in enumerate(top_test_recommendations_with_info, start=1):
    movie_id = prediction.iid
    movie_title = movies_metadata[movies_metadata['id'] == str(movie_id)]['title'].values[0]
    print(f"{i}. {movie_title} (MovieID: {movie_id}, Predicted Rating: {prediction.est:.2f})")


1. Dances with Wolves (MovieID: 581, Predicted Rating: 5.00)
2. Terminator 3: Rise of the Machines (MovieID: 296, Predicted Rating: 5.00)
3. Rebecca (MovieID: 223, Predicted Rating: 5.00)
4. Terminator 3: Rise of the Machines (MovieID: 296, Predicted Rating: 5.00)
5. Flags of Our Fathers (MovieID: 3683, Predicted Rating: 5.00)
6. Pandora's Box (MovieID: 905, Predicted Rating: 5.00)
7. The 39 Steps (MovieID: 260, Predicted Rating: 5.00)
8. License to Wed (MovieID: 2959, Predicted Rating: 5.00)
9. Murder She Said (MovieID: 750, Predicted Rating: 5.00)
10. Confession of a Child of the Century (MovieID: 58559, Predicted Rating: 5.00)


In [ ]:

# Calculate MSE for train set
train_predictions = algo_nmf.test(trainset.build_testset())
train_mse = accuracy.mse(train_predictions)
print(f"MSE for the train set(NMF): {train_mse}")

# Calculate MSE for test set
test_predictions = algo_nmf.test(testset)
test_mse = accuracy.mse(test_predictions)
print(f"MSE for the test set(NMF): {test_mse}")


MSE: 0.4291
MSE for the train set(NMF): 0.42912908405631817
MSE: 0.4320
MSE for the test set(NMF): 0.43203497034285226
